In [89]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import os
import glob

In [90]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [91]:
path = os.getcwd()
extension = 'csv'
os.chdir(path)
files_csv = glob.glob('*.{}'.format(extension))

In [92]:
df_final  = pd.DataFrame(columns=['capital', 'faixa_remuneracao_media_sm', 'analfabeto',
       'ate_5a_incompleto', '5a_completo_fundamental', '6a_a_9a_fundamental',
       'fundamental_completo', 'medio_incompleto', 'medio_completo',
       'superior_incompleto', 'superior_completo', 'mestrado', 'doutorado',
       'sexo','ano'])

In [93]:
def primeiros_tratamentos(df):
    #tirar umas linhas chatas q estao no final do arquivo
    indice_selecoes = df[df['Capital'] == "{ñ class}"].index[0]
    # Apagar as linhas a partir desse índice até o final
    df = df.iloc[:indice_selecoes]
    #tratando top row que esta deslocada
    df.iloc[0, 1:] = df.iloc[0, :-1].values
    #deletando registros que nao tem informações
    df = df[df[' Faixa Remun Média (SM)'] != '{ñ class}']
    #valores NaN em nulo 
    df = df.replace("NaN", np.nan)
    #preenchendo linhas nan para localidade
    df['Capital'] = df['Capital'].ffill()
    #deletando oq vem antes do nome da capital
    df['Capital'] = df['Capital'].str.split(':').str[1]
    #deletar linhas de totais
    df = df[df[' Faixa Remun Média (SM)'] != 'Total']

    #apagar todas as colunas com total no nome
    df = df.drop(df.filter(like='Total').columns, axis=1)

    #corrigindo header
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]
    return df 


In [94]:
def criando_df_feminino(df, ano):
    #criando df feminino
    feminino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Feminino' in coluna]
    df_f=df[feminino_cols]
    #renomeando colunas do que vai ser o novo header
    df_f.iloc[0,0]= "capital"
    df_f.iloc[0,1]= "faixa_remuneracao_media_sm"
    #criando novo header
    new_header = df_f.iloc[0]
    df_f = df_f[1:] 
    df_f.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_f = df_f.loc[:, ~df_f.columns.duplicated()]
    #criand coluna para o sexo feminino
    df_f["sexo"] = "F"
    df_f["ano"] = ano
    #apagar todas as colunas com total no nome
    df_f = df_f.drop(df_f.filter(like='Total').columns, axis=1)
    #corrigindo header
    df_f.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df_f.columns]
    df_f.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df_f.columns]
    return df_f


In [95]:
def criando_df_masculino(df,ano):
    #criando df Masculino
    masculino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Masculino' in coluna]
    df_m=df[masculino_cols]
    #renomeando colunas do que vai ser o novo header
    df_m.iloc[0,0]= "capital"
    df_m.iloc[0,1]= "faixa_remuneracao_media_sm"
    #criando novo header
    new_header = df_m.iloc[0]
    df_m = df_m[1:] 
    df_m.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_m = df_m.loc[:, ~df_m.columns.duplicated()]
    #criand coluna para o sexo masculino
    df_m["sexo"] = "M"
    df_m["ano"] = ano
    #apagar todas as colunas com total no nome
    df_m = df_m.drop(df_m.filter(like='Total').columns, axis=1)
    #corrigindo header
    df_m.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df_m.columns]
    df_m.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df_m.columns]
    return df_m 


In [96]:
for i in files_csv:

    df = pd.read_csv(i,sep=";",  encoding='latin-1',skiprows=1, header=[0])
    ano = re.search(r'\d{4}', i).group()
    df = primeiros_tratamentos(df)
    df_f = criando_df_feminino(df,ano)
    df_m = criando_df_masculino(df,ano)

    df_final = pd.concat([df_final,df_f, df_m])
    
    print(f"File {i}, com shape {df.shape} do ano {ano}, df feminino: {df_f.shape}, df masculino: {df_m.shape} e df final: {df_final.shape}")



#ULTIMOS TRATAMENTOS:
# Remover acentuação
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].apply(lambda x: unidecode(str(x)))

# Aplicar expressão regular para extrair os valores desejados
#df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].apply(lambda x: re.sub(r'^\d+:(.+)', r'\1', str(x)).strip())

#tirar espaco 
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.replace(' ', '_')


# Transformar em minúsculas
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.lower()
df_final.to_csv('escolaridadeXSexoXsalario.csv', index=False)


    

File escolaridadeXSexoXsalarioX2010.csv, com shape (325, 26) do ano 2010, df feminino: (324, 15), df masculino: (324, 15) e df final: (648, 15)
File escolaridadeXSexoXsalarioX2011.csv, com shape (325, 26) do ano 2011, df feminino: (324, 15), df masculino: (324, 15) e df final: (1296, 15)
File escolaridadeXSexoXsalarioX2007.csv, com shape (325, 26) do ano 2007, df feminino: (324, 15), df masculino: (324, 15) e df final: (1944, 15)
File escolaridadeXSexoXsalarioX2013.csv, com shape (325, 26) do ano 2013, df feminino: (324, 15), df masculino: (324, 15) e df final: (2592, 15)
File escolaridadeXSexoXsalarioX2012.csv, com shape (325, 26) do ano 2012, df feminino: (324, 15), df masculino: (324, 15) e df final: (3240, 15)
File escolaridadeXSexoXsalarioX2006.csv, com shape (325, 26) do ano 2006, df feminino: (324, 15), df masculino: (324, 15) e df final: (3888, 15)
File escolaridadeXSexoXsalarioX2016.csv, com shape (325, 26) do ano 2016, df feminino: (324, 15), df masculino: (324, 15) e df fina

/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [97]:
df_final.head()

,capital,faixa_remuneracao_media_sm,analfabeto,ate_5a_incompleto,5a_completo_fundamental,6a_a_9a_fundamental,fundamental_completo,medio_incompleto,medio_completo,superior_incompleto,superior_completo,mestrado,doutorado,sexo,ano
2,Porto Velho - RO,"00:ate_0,50",0,1,3,90,19,135,115,6,24,0,0,F,2010
3,Porto Velho - RO,"01:0,51_a_1,00",7,18,40,76,333,234,2.185,54,126,0,0,F,2010
4,Porto Velho - RO,"02:1,01_a_1,50",84,240,332,1.179,1.937,1.753,14.012,568,531,4,0,F,2010
5,Porto Velho - RO,"03:1,51_a_2,00",33,160,199,406,739,574,8.566,491,1.935,9,2,F,2010
6,Porto Velho - RO,"04:2,01_a_3,00",38,228,187,261,551,329,5.959,482,2.333,14,3,F,2010


In [98]:
#df_melted = pd.melt(df, id_vars=['Capital', ' Faixa Remun Média (SM)'], var_name='Sexo', value_name='Escolaridade')
# df_melted